# Tabluar Prediction

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mli/ag-docs/blob/main/get_started/tabular_quick_start.ipynb)


In a tabular prediction task, we predict the values in a column based on the rest columns' values. This tutorial demonstrates how to use AutoGluon for this task. 

To start, import the {class}`autogluon.tabular.TabularDataset` and 
{class}`autogluon.tabular.TabularPredictor` classes. We will use the former to load data and the latter to train models and predict. 



In [ ]:
#@title Install autogluon
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 203 kB 7.5 MB/s 
     |████████████████████████████████| 48 kB 4.9 MB/s 
     |████████████████████████████████| 141 kB 53.9 MB/s 
     |████████████████████████████████| 272 kB 44.2 MB/s 
     |████████████████████████████████| 59 kB 6.9 MB/s 
     |████████████████████████████████| 61 kB 203 kB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 132 kB 51.5 MB/s 
     |████████████████████████████████| 1.0 MB 42.7 MB/s 
     |████████████████████████████████| 802 kB 41.9 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 54.5 MB 89 kB/s 
     |████████████████████████████████| 1.6 MB 44.7 MB/s 
     |████████████████████████████████| 281 kB 57.3 MB/s 
     |████████████████████████████████| 410 kB 62.4 MB/s 
     |████████████████████████████████| 7

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

The dataset we will use contains individuals' information such as occupation with if or not her income exceeds $50,000, which is the predicting target. We load this dataset directly from a URL. Note that the `TabularDataset` class is a subclass of [pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), any pandas methods can be applied here. 

In [14]:
#url = 'https://autogluon.s3.amazonaws.com/datasets/Inc/'
#train_data = TabularDataset(url+'train.csv')
url = 'https://autogluon.s3.amazonaws.com/openml/sylvine/'
train_data = TabularDataset(url+'train.parquet')
train_data.head()

Loaded data from: https://autogluon.s3.amazonaws.com/openml/sylvine/train.parquet | Columns = 21 / 21 | Rows = 4611 -> 4611


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V12,V13,V14,V15,V16,V17,V18,V19,V20,class
1582,243,205.0,218,430.0,180.0,2619.0,3400.0,9,3005.0,1040.0,...,230,224,2701.0,1597.0,161.0,246,175.0,312.0,155,1
1876,201,47.0,246,5753.0,77.0,1570.0,3285.0,14,2750.0,108.0,...,234,237,900.0,1681.0,0.0,214,194.0,34.0,105,1
1005,232,49.0,241,2736.0,95.0,1961.0,3313.0,34,2193.0,134.0,...,152,250,3278.0,722.0,50.0,170,45.0,87.0,0,1
2659,232,-2.0,241,5342.0,202.0,4166.0,3362.0,25,1974.0,127.0,...,167,200,6062.0,882.0,30.0,253,343.0,322.0,175,1
1295,207,96.0,226,1794.0,114.0,1814.0,3228.0,3,3330.0,42.0,...,200,224,5706.0,2016.0,1.0,236,16.0,67.0,147,1


Our targets are stored in the `class` column, which has two unique values. 



In [15]:
label = 'class'
train_data[label].describe()

count     4611
unique       2
top          0
freq      2306
Name: class, dtype: object

Now construct a `TabularPredictor` instance by specifying the label column name, then train on the dataset with the {func}`autogluon.tabular.TabularPredictor.fit` method. We don't need to specify any other hyperparameters. This method will perform automatic feature engineering, train multiple models, and then ensemble them to form the final predictions. You can find detailed information in the output log.



In [16]:
predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20220709_024326/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220709_024326/"
AutoGluon Version:  0.5.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    4611
Train Data Columns: 20
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['1', '0']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (1) vs negative (0) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor 

The training is often fast, as in default AutoGluon will not try very big models. For example, training the above dataset with ~40K rows should be finished within a few minutes on a normal CPU. But if you want AutoGluon to stop earlier, you can specify the `time_limit` argument in the `fit` method. For example, `fit(..., time_limit=60, ...)` means training at most 1 minute. 

Once training is done, load separate test data to predict.



In [17]:
test_data = TabularDataset(url+'test.parquet')
# Optional: delete the label column for safety check.
y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

Loaded data from: https://autogluon.s3.amazonaws.com/openml/sylvine/test.parquet | Columns = 21 / 21 | Rows = 513 -> 513


263     1
1861    1
4103    1
2293    1
1328    1
Name: class, dtype: object

If you just want to evaluate the model performance, you can call the {func}`autogluon.tabular.TabularPredictor.evaluate` method.

In [18]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9337231968810916,
 'balanced_accuracy': 0.9336545476653697,
 'f1': 0.9360902255639098,
 'mcc': 0.869572145241218,
 'precision': 0.9054545454545454,
 'recall': 0.9688715953307393,
 'roc_auc': 0.9790704036964981}

Now we did a quick through about using AutoGluon for tabular prediction. We used two classes, {class}`autogluon.tabular.TabularDataset` (essentially a pandas DataFrame) to load data and {class}`autogluon.tabular.TabularPredictor` to train (via the `fit` method) and predict (via the `predict` method). You will see similar APIs for other tasks, namely a `Dataset` class to load data and a `Prediction` class to train and predict. 


In addition, AutoGluon simplifies the model training by not requiring feature engineering and specifying model hyperparameters. AutoGluon automatically performs these jobs when running `fit`. You may worry about the resulted longer training time, AutoGluon balances the computational cost and model quality. You can benchmark AutoGluon's performance on the whole dataset loaded above against your favorite machine learning model. But to be fair, you also need to count the time you spend on preprocessing data and tuning your models. 

To know more about AutoGluon, next you can read

- the cheetsheet for a quick overview of the APIs
- tutorials to customize the training and inference
- understand how AutoGluon performs feature engineering and model ensemble. 